<a href="https://colab.research.google.com/github/Somu112/My_DL_practice_step_by_step/blob/main/ResNet_CIFER100.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***RESNET***

In [12]:
import numpy as np

def get_random_eraser(p=0.5, s_l=0.02, s_h=0.4, r_1=0.3, r_2=1/0.3, v_l=0, v_h=255, pixel_level=False):
    def eraser(input_img):
        img_h, img_w, img_c = input_img.shape
        p_1 = np.random.rand()

        if p_1 > p:
            return input_img

        while True:
            s = np.random.uniform(s_l, s_h) * img_h * img_w
            r = np.random.uniform(r_1, r_2)
            w = int(np.sqrt(s / r))
            h = int(np.sqrt(s * r))
            left = np.random.randint(0, img_w)
            top = np.random.randint(0, img_h)

            if left + w <= img_w and top + h <= img_h:
                break

        if pixel_level:
            c = np.random.uniform(v_l, v_h, (h, w, img_c))
        else:
            c = np.random.uniform(v_l, v_h)

        input_img[top:top + h, left:left + w, :] = c

        return input_img

    return eraser

In [13]:
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input

import tensorflow.keras as keras
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, UpSampling2D, Flatten, BatchNormalization, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras import optimizers
from keras.datasets import cifar100
import tensorflow as tf
from keras.utils import np_utils
import numpy as np
import matplotlib.pyplot as plt
import time
from skimage.transform import resize
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras_preprocessing.image import ImageDataGenerator

In [14]:
num_classes = 100
np_epochs = 100
(x_train, y_train), (x_test, y_test) = cifar100.load_data()

x_train = preprocess_input(x_train)
x_test  = preprocess_input(x_test)

datagen = ImageDataGenerator(preprocessing_function=get_random_eraser(v_l=0, v_h=1, pixel_level=True))
datagen.fit(x_train)

In [15]:
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)

In [16]:
resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(256, 256, 3))


for layer in resnet_model.layers:
  if isinstance(layer, BatchNormalization):
    layer.trainable = True
  else:
    layer.trainable = False

model = Sequential()
model.add(UpSampling2D())
model.add(UpSampling2D())
model.add(UpSampling2D())
model.add(resnet_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(256, activation='relu'))
model.add(Dropout(.25))
model.add(BatchNormalization())
model.add(Dense(num_classes, activation='softmax'))

In [17]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [18]:
t=time.time()

historytemp = model.fit(datagen.flow(x_train, y_train,
                                     batch_size=16),
                        steps_per_epoch=100,
                        epochs=np_epochs,
                        validation_data=(x_test, y_test))

print('Traning time: %s' % (t - time.time()))

Epoch 1/100
100/100 [==============================] - 117s 1s/step - loss: 4.3921 - accuracy: 0.0694 - val_loss: 3.8741 - val_accuracy: 0.1135
Epoch 2/100
100/100 [==============================] - 107s 1s/step - loss: 3.6333 - accuracy: 0.1525 - val_loss: 3.1726 - val_accuracy: 0.2155
Epoch 3/100
100/100 [==============================] - 108s 1s/step - loss: 3.2393 - accuracy: 0.2100 - val_loss: 2.7247 - val_accuracy: 0.3269
Epoch 4/100
100/100 [==============================] - 108s 1s/step - loss: 2.9492 - accuracy: 0.2831 - val_loss: 2.3591 - val_accuracy: 0.3927
Epoch 5/100
100/100 [==============================] - 108s 1s/step - loss: 2.8028 - accuracy: 0.3063 - val_loss: 2.1559 - val_accuracy: 0.4290
Epoch 6/100
100/100 [==============================] - 107s 1s/step - loss: 2.6322 - accuracy: 0.3356 - val_loss: 2.0372 - val_accuracy: 0.4577
Epoch 7/100
100/100 [==============================] - 107s 1s/step - loss: 2.4550 - accuracy: 0.3812 - val_loss: 1.8987 - val_accuracy:

In [19]:
!nvidia-smi

Tue Aug 17 13:28:15 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P0    67W / 149W |   8442MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------